In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px

### Read-In Data

In [ ]:
data_raw = pd.read_csv("../data/census.csv")

In [ ]:
data_raw.head(5)

In [ ]:
# Remove Space of column name
data_raw.columns = [col.strip() for col in data_raw.columns]

### Check Data Type and Missing Data

In [ ]:
df_data_type = pd.DataFrame(data_raw.dtypes)
df_data_type.reset_index(inplace = True)
df_data_type.rename(columns = {'index' : "columns_name", 0: "Data Type"},inplace = True)

df_data_missing = pd.DataFrame(data_raw.isna().mean()*100)
df_data_missing.reset_index(inplace = True)
df_data_missing.rename(columns = {'index' : "columns_name", 0: "Missing Percentage"},inplace = True)

df_data_checking = pd.merge(df_data_type, df_data_missing, on = 'columns_name')

In [ ]:
df_data_checking

### Data Check by Columns

#### Categorical Data

In [ ]:
categorical_columns = data_raw.select_dtypes("object").columns
numerical_columns = list(set(data_raw.columns).difference(set(categorical_columns)))

In [ ]:
def plot_categorical_count(df:pd.DataFrame, column_count:str) -> None:
    count_df = pd.DataFrame(df.groupby([column_count]).size()).reset_index()
    count_df.rename(columns = {0 : 'Records Count'}, inplace = True)
    count_df.sort_values(by = 'Records Count', ascending= False, inplace = True)

    count_plot = px.histogram(count_df, x = column_count , y = "Records Count",
                              color = column_count, width= 700, height=400,
                              title= f"Count of {column_count}")
    count_plot.show()

In [ ]:
for col_name in categorical_columns:
    plot_categorical_count(data_raw, col_name)

### Data Cleaning - Categorical Data

In [ ]:
# Replace '?' with np.nan to make replace with imputed value
data_raw.replace('?', np.nan, inplace = True)
data_raw.replace('? ', np.nan, inplace = True)
data_raw.replace(' ?', np.nan, inplace = True)

# Replace all NA with mode on each columns
# Remove white space in data
for cat_col in categorical_columns:
    col_mode = data_raw[cat_col].mode()
    data_raw[cat_col] = data_raw[cat_col].str.strip()
    data_raw[cat_col] = data_raw[cat_col].fillna(str(col_mode))

### Categorical Grouping

In [ ]:
# Group naitve-country into "United-State" and "Other"
data_raw['native-country'] = np.where(data_raw['native-country'] != 'United-States', 'Other', 'United-States')

# Group race into "White", "Black" and "Other"
data_raw['race'] = np.where((data_raw['race'] == 'Asian-Pac-Islander') |
                            (data_raw['race'] == 'Amer-Indian-Eskimo'), 'Other', data_raw['race'])

data_raw['education'].replace(['11th', '9th', '7th-8th', '5th-6th', 
                               '10th', '1st-4th', 'Preschool', '12th'], 'School', inplace = True)                       

#### Numerical Data

In [ ]:
def numerical_hist_plot(data:pd.DataFrame, numerical_col:str) -> None:
    histogram_plot = px.histogram(data, x = numerical_col,
                                  width = 700, height=400,
                                  title= f"Hitogram of {numerical_col}")
    histogram_plot.show()

In [ ]:
for col_name in numerical_columns:
    numerical_hist_plot(data_raw, col_name)

In [ ]:
data_raw.drop(columns = ['capital-gain', 'capital-loss'], inplace = True)

In [ ]:
data_raw.to_csv("../data/census_clean.csv", index = 0)